# Creation of H2 Dataset

In order to more accurately compare the HGG data to the older D data, it is necessary to reduce the resolution of the data -- converting from gridbox sizes of 1x1 to 2x2. The D data has gridbox sizes of 2.5x2.5, however 2x2 is accurate enough. The reduced resolution dataset will be dubbed H2 and will be stored in the exact same way as the H1 dataset but with a shape of (12,12,180*90)

# Description of Creation Process

The mathematics for determining WS of each gridbox is going to be the same as for the H1 dataset. What needs elaboration is the reduction is resolution.

### How Resolution is Reduced
Consider a (360, 180) WS map created from the H1 data. Each one of the 64800 values is a single pixel representing the WS count for a gridbox of resolution 1x1 degrees. To reduce the resolution of this map to (180, 90) one could add together quadrants of pixels. Starting with the pixel located at [0,0], consider the three adjacent pixels: one along the same row (pixel [0,1]), another along the same column (pixel [1,0]), and the third along the diagonal (pixel [1,1]). These four pixels can be added together to make one larger pixel located at [0,0] on the (180,90) map. 

The plan is to convert non-overlapping adjancent quadrants into single pixels. For example, the adjacent pixel along the same row of the reduced (180,90) map will be composed of the pixels located at [0,2],[0,3],[1,2], and [1,3] on the higher resolution (360,180) map. Keep traversing along the same row until the boundary of the map is reached and do the same along the next appropriate rows. That is, rows 3 and 4 will be used for the next row of quadrants (skip two rows to prevent overlapping). 

### Prepping Data for Resolution Reduction

The algorithm laid out above to reduce the resolution of a map is what will be followed -- however, it is not the resolution of WS maps from H1 data that needs to be reduced. The idea is to reduce the resolution of the PCTAU histograms from the n_pctaudist variable and then compute the appropriate WS for each gridbox with these new 2x2 degree gridboxes. 
To reduce the resolution of the gridboxes, the data must be prepped in the following way:

1. Convert n_pctaudist, ntotal, and the mask of n_pctaudist to Equal-Angle.
2. For each PCTAU histogram in n_pctaudist, flatten it to a 1D array of length 42.
3. Arrange the flattened histograms in the appropriate order used to create a (360,180) map. This will result in a matrix of shape (360, 180, 42) in which each [x,y,:] is a flattened gridbox.
4. At this point, each slice along axis=2 is a (360,180) map of the same cell of all PCTAU histograms. So, this means the first slice is composed entirely of the [0,0] pixel of every PCTAU histogram, the second slice composed of the [0,1] pixel, etc.
5. Do the same for the mask of n_pctaudist
6. Convert Equal-Angle ntotal to a (360,180) map and copy it into a (360,180,42) matrix to make normalization easier.

### Calculating WS for each Gridbox

All data has been reshaped to form matricies of shape (360, 180, 42). Now all that is left to do is reduce the resolution of each of the 42 (360,180) slices of each matrix. This will produce new matricies of shape (180,90,42). Next, the WS for each gridbox -- where a single gridbox is [x,y,:] of the reshaped and reduced n_pctaudist matrix -- is calculated with the results arranged in the same exact way as the H1 data. The only difference is the H2 data will be of shape (12,12,16200) instead of (12,12,64800) because the resolution was cut in half.

### Code for Resolution Reduction

The sheer amount of data reshaping will cause the H2 dataset creation time to be quite large if done in Python. As a result, these steps have been implemented in Fortran. Please see the subroutines with accompanying descriptions in the file **analysis2.f90**.

In [3]:
import os
import gc
import time
import netCDF4
import analysis2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
hgg = '/run/media/dtropf/Seagate Backup Plus Drive/hgg_data/'
files = []
yearlist = []
years = [str(i) for i in range(1984, 2013)]
for year in years:
    yearpath = hgg+year
    yearlist = []
    for mpath in sorted(os.listdir(yearpath)):
        dpath = yearpath+'/'+mpath
        yearlist.append(dpath)
    files.append(yearlist)

In [ ]:
weather_states = np.loadtxt(os.getcwd()+'/'+'pctau.txt').reshape(12, 7, 6).transpose((0,2,1))/100.0
wsnums = np.zeros((180, 90, 12))
save_path = '/home/dtropf/Documents/NASA/clouds/validation/test_data/'
t3 = time.time()
for year, yp in enumerate(files):
    t1 = time.time()
    annual_array = np.zeros((12,12,180*90))
    for i, mp in enumerate(yp):
        for df in sorted(os.listdir(mp)):
            path = mp+'/'+df
            data = netCDF4.Dataset(path)
            pctaudist = data['n_pctaudist'][:,:,:]
            sqlonbeg = data['sqlon_beg'][:]
            sqlonend = data['sqlon_end'][:]
            eqlat = data['eqlat_index'][:]
            ntotal = data['n_total'][:]
            pctau, mask, ntot = analysis2.reshape_data(pctaudist, pctaudist.mask, ntotal, sqlonbeg, sqlonend, eqlat)
            pctau_reduce = analysis2.reduce_pctau(pctau, mask)
            mask_reduce = analysis2.reduce_mask(mask)
            ntot_reduce = analysis2.reduce_ntotal(ntot, mask)
            pctau_norm = analysis2.fnorm(pctau_reduce, ntot_reduce, mask_reduce)
            wstmp = analysis2.findws(pctau_norm, weather_states, mask_reduce)
            wsnums[:,:,:] = wsnums[:,:,:] + wstmp[:,:,:]
            for j in range(12):
                annual_array[i,j,:] = annual_array[i,j,:] + wstmp[:,:,j].flatten()
            del pctaudist, sqlonbeg, sqlonend, eqlat, ntotal, data
            gc.collect()
        print('Finished month: ', mp)
    #np.save(save_path+'H2_'+years[year], annual_array)
    t2 = time.time()
    print('Time for ', years[year], t2-t1)
t4 = time.time()
print('Time for Total: ', t4-t3)